In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
with open('healthcare-claims-prediction_kaggel/patient_data_train.json', 'r') as file:
    train_data_json = json.load(file)
with open('healthcare-claims-prediction_kaggel/patient_data_test.json', 'r') as file:
    test_data_json = json.load(file)
train_claims = pd.read_csv('healthcare-claims-prediction_kaggel/train.csv')

def normalize_and_clean_data(data):
    '''
    Normalizing and handling the data
    '''
    import warnings
    warnings.filterwarnings("ignore")
    df = pd.json_normalize(data)

    # each patient is unique record,i will be replacing nulls with 0 values

    # so First we will do this for the features related to past years
    df["Out patient costs.2023"].fillna(0,inplace=True)

    df["Out patient costs.2022"].fillna(0,inplace=True)

    df["Out patient costs.2021"].fillna(0,inplace=True)

    df["Out patient costs.2020"].fillna(0,inplace=True)

    df["Out patient costs.2019"].fillna(0,inplace=True)

    df["Conditions.HD"].fillna(0,inplace=True)

    df["Conditions.HT"].fillna(0,inplace=True)

    df["Conditions.DB"].fillna(0,inplace=True)

    df["Conditions.AT"].fillna(0,inplace=True)

    return df

train_data = normalize_and_clean_data(train_data_json)
test_data = normalize_and_clean_data(test_data_json)
    # As we already know, we have only 5 past years to handle the values
    # Now, we need to do the similar kind of analysis for conditions features as well.

train_data = pd.merge(train_data, train_claims, on='PatientID')

trian_ID_col = train_data[["PatientID"]]
train_data.drop(["PatientID"],axis=1,inplace=True)
test_ID_col = test_data[["PatientID"]]
test_data.drop(["PatientID"], axis=1,inplace=True)


from sklearn.preprocessing import LabelEncoder

sc = LabelEncoder()
sc.fit(train_data["Sex"])
train_data["Sex"] = sc.transform(train_data["Sex"])
test_data["Sex"] = sc.transform(test_data["Sex"])

x = train_data.drop(["TotalClaims"],axis=1)
y = train_data[["TotalClaims"]]

# These lines are making both the features in simlar order

temp = test_data[["Conditions.AT","Conditions.DB"]]
test_data.drop(["Conditions.AT","Conditions.DB"], axis=1, inplace=True)
test_data["Conditions.DB"] = temp["Conditions.DB"]
test_data["Conditions.AT"] = temp["Conditions.AT"]

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

# RFE for feature selection
selector = RFE(estimator=rf, n_features_to_select=8)
selector.fit(x_train, y_train)

selected_features = x.columns[selector.support_]
print("Selected features:", selected_features)

# Hyperparameter tuning using Grid Search

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x[selected_features],y, test_size=0.3, random_state=42)
test_data = test_data[selected_features]






Selected features: Index(['Age', 'Conditions.HD', 'Conditions.HT', 'Out patient costs.2023',
       'Out patient costs.2022', 'Out patient costs.2021', 'Conditions.DB',
       'Conditions.AT'],
      dtype='object')


In [21]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
rf = RandomForestRegressor(random_state=42)


# Sclaing the data

from sklearn.preprocessing import StandardScaler

sc= StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
test_data = sc.transform(test_data)

param_grid_xgb = {
    'n_estimators': [100, 200, 300, 500],   
    'learning_rate': [0.01, 0.1, 0.2, 0.001],
    'max_depth': [3, 5, 7, 9],         
    'subsample': [0.8, 1.0],           
    'colsample_bytree': [0.8, 1.0]     
}


xgb = XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42)

grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb,
                               scoring='neg_mean_absolute_error', cv=3,
                               verbose=1, n_jobs=-1)

grid_search_xgb.fit(x_train, y_train)

best_xgb_model = grid_search_xgb.best_estimator_

pred_train = best_xgb_model.predict(x_train)
pred_test = best_xgb_model.predict(x_test)

train_mae = mean_absolute_error(y_train, pred_train)
test_mae = mean_absolute_error(y_test, pred_test)

print(f"XGBoost Train MAE: {train_mae}")
print(f"XGBoost Test MAE: {test_mae}")


Fitting 3 folds for each of 256 candidates, totalling 768 fits
XGBoost Train MAE: 245.29246206283568
XGBoost Test MAE: 982.0578189970665


In [23]:
res = best_xgb_model.predict(test_data)
test_ID_col["TotalClaims"] = res
test_ID_col.to_csv("submission.csv" , index = False)